In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install accelerate

In [ ]:
pip install datasets

In [ ]:
import transformers
import torch
import json
from datasets import load_dataset
dataset = load_dataset("avnishkr/topics_in_ai")
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [ ]:
def response(text):

  messages = [
      {"role": "system", "content": "You have to give short answer for some open-ended question"},
      {"role": "user", "content": text},
  ]
  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
      num_return_sequences = 1
  )
  return outputs[0]["generated_text"][len(prompt):]

In [ ]:
generated_responses = [{dataset['test']['question'][i]: response(dataset['test']['question'][i])} for i in range(dataset.num_rows['test'])]

In [ ]:
# Save the dictionary as JSON into a file
with open("/content/drive/MyDrive/topics_in_ai/generated_responses.json", "w") as f:
    json.dump(generated_responses, f, indent=4)

In [ ]:
generated_responses